In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
pd.options.display.float_format = '{:.4f}'.format
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
%load_ext autoreload
%autoreload 2
%matplotlib inline

from perturbseq import *

# Load Perturbseq experiment (run this section once)

*The raw UMI threshold here is used to remove cell barcodes from the raw gene expression matrix that have too few UMIs to be useful. (Loading the whole matrix into memory at some point becomes wasteful when most entries are zero.) This should be set to the same threshold as used in the guide barcode calling script as guide identities will not be available for any cell barcode below the threshold set there.*

**Note that we did not use cellranger's cell barcode calling since in a heterogeneous population their heuristic tends to remove cells bearing perturbations that induce low UMI counts. This was addressed by a new algorithm in cellranger 3.0**

In [3]:
pop = CellPopulation.from_file('/home/tmn/sequencing/gi/gi', filtered=False, raw_umi_threshold=2000)

# This guide lands between RHOXF2 and RHOXF2B but appears to have stronger effect on RHOXF2B
pop.cells['guide_identity'] = pop.cells['guide_identity'].map(lambda x: x.replace('RHOXF2', 'RHOXF2B'))
pop.to_hdf('/home/tmn/sequencing/gi/gi/outs/final_pop.hdf')

Loading digital expression data: /home/tmn/sequencing/gi/gi/outs/raw_gene_bc_matrices_mex/GRCh38/matrix.mtx...
Densifying matrix...
Filtering cell barcodes with fewer than 2000 UMIs...
Loading guide identities:/home/tmn/sequencing/gi/gi/outs/raw_cell_identities.csv...
Generating summary statistics...
Done.
Writing matrix...
Writing metadata...
Done in 39.5335490704s.


**Remove multiplets**

In [4]:
pop = pop.subpopulation(cells='single_cell')
pop.to_hdf('/home/tmn/sequencing/gi/gi/outs/final_single_cell_pop.hdf')

Generating summary statistics...
Done.
Writing matrix...
Writing metadata...
Done in 27.1661679745s.


**Add useful metadata**

In [5]:
# these have different names in the emap and in GChr38
name_replacer = {'C3orf72': 'FOXL2NB',
                 'C19orf26': 'CBARP',
                 'KIAA1804': 'RP5-862P8.2',
                 'RHOXF2': 'RHOXF2B',
                }

inverse_name_replacer = {v: k for k, v in name_replacer.iteritems()}

In [6]:
pop.cells['guide_target'] = pop.cells['guide_identity'].map(lambda x: x.split('__')[0] if isinstance(x, basestring) else 'no_id')
pop.cells['num_targets'] = pop.cells['guide_target'].map(lambda x: len(x.split('_')))
pop.cells['guide_target_1'] = pop.cells['guide_target'].map(lambda x: x.split('_')[0])
pop.cells['guide_target_2'] = ''
pop.cells.loc[pop.cells['num_targets'] == 2, 'guide_target_2'] = pop.cells.loc[pop.cells['num_targets'] == 2, 'guide_target'].map(lambda x: x.split('_')[1])
pop.cells['guide_target_1'] = pop.cells['guide_target_1'].map(lambda x: name_replacer.get(x, x))
pop.cells['guide_target_2'] = pop.cells['guide_target_2'].map(lambda x: name_replacer.get(x, x))
pop.cells['guide_target'] = pop.cells['guide_target_1'] + '_' + pop.cells['guide_target_2']

control_targets = np.setdiff1d(pop.cells.groupby('guide_target').count().sort_values('number_of_cells', ascending=False).head(4).index.values,
                               'NegCtrl1_NegCtrl0')
pop.cells['perturbed'] = pop.cells['guide_target'].map(lambda x: 'control' if x in control_targets else x)
pop.cells.query('perturbed == "control"').groupby('gem_group').count()

,guide_identity,guide_read_count,guide_UMI_count,guide_coverage,gemgroup,good_coverage,number_of_cells,cellranger_called,guide_target,single_cell,UMI_count,num_targets,guide_target_1,guide_target_2,perturbed
gem_group,,,,,,,,,,,,,,,
1,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008,1008
2,928,928,928,928,928,928,928,928,928,928,928,928,928,928,928
3,863,863,863,863,863,863,863,863,863,863,863,863,863,863,863
4,874,874,874,874,874,874,874,874,874,874,874,874,874,874,874
5,821,821,821,821,821,821,821,821,821,821,821,821,821,821,821
6,976,976,976,976,976,976,976,976,976,976,976,976,976,976,976
7,934,934,934,934,934,934,934,934,934,934,934,934,934,934,934
8,949,949,949,949,949,949,949,949,949,949,949,949,949,949,949


**For memory efficiency, remove genes with no counts from expression matrix.**

In [7]:
strip_low_expression(pop)

**Generate normalized matrix**

In [8]:
pop.normalized_matrix = normalize_to_gemgroup_control(pop, control_cells='perturbed == "control"')

Normalizing all cells to 14955.0 UMI...
groupby: index in @key_barcodes and (perturbed == "control") (key = gem_group)
Processing gem group 1
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.
4.39555191994
Processing gem group 2
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.
4.0634598732
Processing gem group 3
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.
4.69665813446
Processing gem group 4
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.
3.79387998581
Processing gem group 5
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to 

In [9]:
control_targets = np.setdiff1d(pop.cells.groupby('guide_target').count().sort_values('number_of_cells', ascending=False).head(4).index.values,
                                'NegCtrl1_NegCtrl0')

control_pop = pop.subpopulation(cells='perturbed == "control"', normalized_matrix='inherit')

Generating summary statistics...
Done.
Inheriting from parent normalized matrix...


**Add cell cycle scores**

In [10]:
cell_cycle_genes = get_cell_phase_genes(control_pop, refine=True, threshold=0.3)

perturbseq/cell_cycle.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  expression_matrix['total'] = expression_matrix.mean(axis=1)


In [11]:
add_cell_cycle_scores(pop, gene_list=cell_cycle_genes)
add_cell_cycle_scores(control_pop, gene_list=cell_cycle_genes)

In [12]:
control_pop.to_hdf('/home/tmn/sequencing/gi/gi/outs/control_pop.hdf', store_normalized_matrix=True)

Writing matrix...
Writing normalized matrix...
Writing metadata...
! Converting categorical columns to string...
Done in 3.15682196617s.


**Save as hdf5 so that we don't have to do slow steps every time notebook runs**

In [13]:
pop.to_hdf('/home/tmn/sequencing/gi/gi/outs/final_single_cell_pop_with_normalized.hdf', store_normalized_matrix=True)

Writing matrix...
Writing normalized matrix...
Writing metadata...
! Converting categorical columns to string...
Done in 103.400902987s.


# Load pre-processed Perturbseq experiment from hdf5

In [14]:
# pop = CellPopulation.from_hdf('/home/tmn/sequencing/gi/gi/outs/final_single_cell_pop_with_normalized.hdf')

In [15]:
pop.info()

Matrix
<class 'pandas.core.frame.DataFrame'>
Index: 93658 entries, AAACCTGAGGCATGTG-1 to TTTGTCATCTGGCGAC-8
Columns: 24665 entries, ENSG00000243485 to ENSG00000271254
dtypes: int32(24665)
memory usage: 8.6 GB

Normalized matrix
<class 'pandas.core.frame.DataFrame'>
Index: 93658 entries, AAACCTGAGGCATGTG-1 to TTTGTCATCTGGCGAC-8
Columns: 24665 entries, ENSG00000243485 to ENSG00000271254
dtypes: float64(24665)
memory usage: 17.2 GB


# Generate mean population

In [16]:
mean_pop = pop.average('guide_target', show_progress=True)

Computing average expression matrices...


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))


Computing normalized average expression matrices...


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))


Computing clusters...


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))


Generating summary statistics...
Done.


In [17]:
# for expression comparisons we will just use average RNAseq profiles normalized to same UMI count
# mean_pop.normalized_matrix = equalize_UMI_counts(mean_pop.matrix,
#                                                  median_umi_count=pop.cells.query('perturbed=="control"')['UMI_count'].median())

**Add metadata**

In [18]:
mean_pop.cells['first_target'] = mean_pop.cells.index.map(lambda x: x.split('_')[0])
mean_pop.cells['second_target'] = mean_pop.cells.index.map(lambda x: x.split('_')[1])
mean_pop.cells['guide_target'] = mean_pop.cells.index
mean_pop.cells['num_targets'] = 2 - mean_pop.cells.index.str.count('NegCtrl')
mean_pop.cells['first_id'] = mean_pop.cells['first_target'].map(lambda x: mean_pop.gene_ids(x))
mean_pop.cells['second_id'] = mean_pop.cells['second_target'].map(lambda x: mean_pop.gene_ids(x))

mean_pop.add_property(cells=mean_pop.metaapply({'first_expr': lambda meta, expr: expr.loc[meta.name, meta['first_id']],
                                                 'second_expr': lambda meta, expr: expr.loc[meta.name, meta['second_id']],
                                                }))

mean_pop.add_property(cells=metaapply(mean_pop.cells, control_pop.genes,
                      {'control_first_expr': lambda meta, expr: expr.loc[meta['first_id'], 'mean'],
                       'control_second_expr': lambda meta, expr: expr.loc[meta['second_id'], 'mean'],
                      }))

mean_pop.cells['fold_first_expr'] = mean_pop.cells['first_expr']/mean_pop.cells['control_first_expr']
mean_pop.cells['fold_second_expr'] = mean_pop.cells['second_expr']/mean_pop.cells['control_second_expr']
mean_pop.cells['log_fold_first_expr'] = np.log10(mean_pop.cells['fold_first_expr'])
mean_pop.cells['log_fold_second_expr'] = np.log10(mean_pop.cells['fold_second_expr'])
mean_pop.cells['diff_first_expr'] = mean_pop.cells['first_expr'] - mean_pop.cells['control_first_expr']
mean_pop.cells['diff_second_expr'] = mean_pop.cells['second_expr'] - mean_pop.cells['control_second_expr']
mean_pop.cells['pct_first_expr'] = mean_pop.cells['diff_first_expr']/mean_pop.cells['control_first_expr']
mean_pop.cells['pct_second_expr'] = mean_pop.cells['diff_second_expr']/mean_pop.cells['control_second_expr']
mean_pop.cells['log_control_first_expr'] = np.log10(mean_pop.cells['control_first_expr'])
mean_pop.cells['log_control_second_expr'] = np.log10(mean_pop.cells['control_second_expr'])

perturbseq/cell_population.py:1197: UserWarning: Function application containing missing keys that were replaced by nans.
  warnings.warn('Function application containing missing keys that were replaced by nans.')
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log10


# Differential expression

In [19]:
ks, ps, adj_ps = ks_de(pop,
                       key='guide_target',
                       control_cells='perturbed == "control"',
                       genes='mean > 0.25',
                       normalized=True,
                       alpha=0.001,
                       n_jobs=24)

7353 control cells
groupby: index in @key_barcodes (key = guide_target)


[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    7.8s
[Parallel(n_jobs=24)]: Done  13 tasks      | elapsed:   13.5s
[Parallel(n_jobs=24)]: Done  24 tasks      | elapsed:   19.4s
[Parallel(n_jobs=24)]: Done  37 tasks      | elapsed:   26.7s
[Parallel(n_jobs=24)]: Done  50 tasks      | elapsed:   33.4s
[Parallel(n_jobs=24)]: Done  65 tasks      | elapsed:   42.0s
[Parallel(n_jobs=24)]: Done  80 tasks      | elapsed:   49.6s
[Parallel(n_jobs=24)]: Done  97 tasks      | elapsed:   59.3s
[Parallel(n_jobs=24)]: Done 114 tasks      | elapsed:  1.1min
[Parallel(n_jobs=24)]: Done 133 tasks      | elapsed:  1.3min
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:  1.5min
[Parallel(n_jobs=24)]: Done 173 tasks      | elapsed:  1.7min
[Parallel(n_jobs=24)]: Done 194 tasks      | elapsed:  1.9min
[Parallel(n_jobs=24)]: Done 217 tasks      | elapsed:  2.1min
[Parallel(n_jobs=24)]: Done 240 tasks      | elapsed:  2.3min
[Parallel(n_jobs=24)]: Done 269 out of 287 | elapsed:  2.6min remainin

In [20]:
mean_pop.cells['ks_de'] = (adj_ps < 0.001).sum()

# Emap processing

In [113]:
emap = pd.read_csv('./final_emaps/filter15/CRISPRa_K562_emap_gene.txt', index_col=0, sep='\t')
phen_matrix_guide = pd.read_csv('./final_emaps/filter15/CRISPRa_K562_replicateAverage_phenotypeMatrix_abbaAveraged.txt', index_col=0, sep='\t')
single_phen = pd.read_csv('./final_emaps/filter15/CRISPRa_K562_replicateAverage_singlePhenotypes_abbaAveraged.txt', index_col=0, sep='\t')
guide_phen_matrix = pd.read_csv('./final_emaps/filter15/CRISPRa_K562_replicateAverage_phenotypeMatrix_abbaAveraged.txt', index_col=0, header=0, sep='\t')

In [116]:
# averaging guide phenotypes targeting the same gene
phen_matrix = phen_matrix_guide.copy()
phen_matrix['gene'] = phen_matrix.index.map(lambda x: x.split('_')[0])
phen_matrix = phen_matrix.groupby('gene').mean()
phen_matrix = phen_matrix.T
phen_matrix['gene'] = phen_matrix.index.map(lambda x: x.split('_')[0])
phen_matrix = phen_matrix.groupby('gene').mean()

In [117]:
# get the specific guides used in Perturbseq, which only uses one guide per gene instead of two
guide_names = pd.read_excel('./final_emaps/CRISPRa_epistasis_GBC_final_corrected.xlsx')
max_guide_names = pd.read_csv('./final_emaps/20160125_CRISPRa_doubles_oligos.txt', sep='\t', index_col=[0, 1])

single_guides = guide_names.query('gene_B == "NegCtrl0"')['protospacer_sequence_A']
max_guide_names['protospacer sequence'] = max_guide_names['protospacer sequence'].str.upper()
chosen_guides = max_guide_names[max_guide_names['protospacer sequence'].isin(single_guides)].reset_index() \
                        .rename(columns = {'level_0': 'guide_id',
                                           'protospacer sequence': 'protospacer'})
chosen_guides = chosen_guides[['gene', 'guide_id', 'protospacer']].set_index('gene')
chosen_guides = chosen_guides.loc[chosen_guides.index != 'RHOXF2B']
chosen_guides.to_csv('./final_emaps/20180821_guides_chosen_for_perturbseq.csv')

In [121]:
# name differences between Max's guide names and GRCh38
emap.columns = emap.columns.map(lambda x: name_replacer.get(x, x))
emap.index = emap.index.map(lambda x: name_replacer.get(x, x))

phen_matrix.columns = phen_matrix.columns.map(lambda x: name_replacer.get(x, x))
phen_matrix.index = phen_matrix.index.map(lambda x: name_replacer.get(x, x))

In [122]:
single_phen['guide_target'] = single_phen.index.map(lambda x: x.split('_')[0])
single_phen['guide_target'] = single_phen['guide_target'].map(lambda x: name_replacer.get(x, x))

single_fitnesses = single_phen.groupby('guide_target').mean()['b.mean']
single_fitnesses.index = single_fitnesses.index.map(lambda x: name_replacer.get(x, x))
single_fitnesses = pd.DataFrame(single_fitnesses)
single_fitnesses = single_fitnesses.rename(columns={'b.mean': 'average_fitness'})

chosen_fitness = unfiltered_single_phen.loc[chosen_guides['guide_id']].rename(columns={'b.mean': 'chosen_guide_fitness'})['chosen_guide_fitness']
chosen_fitness.index = chosen_fitness.index.map(lambda x: name_replacer.get(x.split('_')[0], x.split('_')[0]))
single_fitnesses['chosen_guide_fitness'] = chosen_fitness

In [124]:
# this table is needed for running Max's emap scripts
single_phen['gene'] = single_phen.index.map(lambda x: x.split('_')[0]).map(lambda x: name_replacer.get(x, x))
single_phen_gene = single_phen.groupby('gene').mean().rename(columns={'a.mean': 'mean'})['mean']
single_phenotypes = pd.DataFrame(single_phen_gene).rename(columns={'mean': 'a.mean'})
single_phenotypes['b.mean'] = single_phen_gene

unfiltered_single_phenotypes = single_fitnesses[['chosen_guide_fitness']].rename(columns={'chosen_guide_fitness': 'a.mean'})
unfiltered_single_phenotypes['b.mean'] = unfiltered_single_phenotypes['a.mean']

In [125]:
# fitness mapper, maps names to raw fitness values
fitness_mapper = dict()
unfiltered_fitness_mapper = dict()

for perturbation in pop.cells['guide_target'].unique():
    first, second = perturbation.split('_')
    try:
        if 'NegCtrl' in second:
            fitness_mapper[perturbation] = single_fitnesses.loc[first, 'average_fitness']
            unfiltered_fitness_mapper[perturbation] = single_fitnesses.loc[first, 'chosen_guide_fitness']
        elif 'NegCtrl' in first:
            fitness_mapper[perturbation] = single_fitnesses.loc[second, 'average_fitness']
            unfiltered_fitness_mapper[perturbation] = single_fitnesses.loc[second, 'chosen_guide_fitness']
        else:
            fitness_mapper[perturbation] = phen_matrix.loc[first, second]
            unfiltered_fitness_mapper[perturbation] = perturbseq_phen_matrix.loc[first, second]
    except KeyError:
        fitness_mapper[perturbation] = np.nan
        unfiltered_fitness_mapper[perturbation] = np.nan

In [126]:
mean_pop.cells['fitness'] = mean_pop.cells.index.map(lambda x: fitness_mapper[x])
mean_pop.cells['perturbseq_fitness'] = mean_pop.cells.index.map(lambda x: unfiltered_fitness_mapper[x])

In [127]:
emap.to_csv('./final_emaps/processed_emap.csv')
phen_matrix.to_csv('./final_emaps/processed_fitnesses.csv')
single_fitnesses.to_csv('./final_emaps/single_fitnesses.csv')
single_phenotypes.to_csv('./final_emaps/single_phenotype_table.csv')

# Save mean population

In [76]:
mean_pop.to_hdf('/home/tmn/sequencing/gi/gi/outs/mean_pop_with_normalized.hdf', store_normalized_matrix=True)

Writing matrix...
Writing normalized matrix...
Writing metadata...
Done in 0.156399965286s.


In [77]:
mean_pop.matrix.to_csv('./data_sharing/mean_expression_unnormalized.csv')
mean_pop.normalized_matrix.to_csv('./data_sharing/mean_expression_normalized.csv')

# Create mean population of only singles present in both emap and perturbseq

In [78]:
single_genes = np.intersect1d(reduce(np.union1d, mean_pop.cells.query('num_targets == 1')['guide_target'].map(lambda x: x.split('_'))),
               emap.columns.map(lambda x: name_replacer.get(x, x)))
# we know these don't work -- dropped out due to toxicity
single_genes = np.setdiff1d(single_genes, ['BAK1', 'BCL2L11'])

# maps gene name to single (SLC38A2 is the lone exception where we don't have it in position 1)
single_gene_perturbations = dict()

for gene in single_genes:
    if gene + '_NegCtrl0' in mean_pop.cells.index:
        single_gene_perturbations[gene] = gene + '_NegCtrl0'
    elif 'NegCtrl0_' + gene in mean_pop.cells.index:
        single_gene_perturbations[gene] = 'NegCtrl0_' + gene

In [79]:
single_mean_pop = pop.average('guide_target', cells='guide_target in @singles',
                              singles=single_gene_perturbations.values(),
                              show_progress=True)

Computing average expression matrices...


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))

groupby: index in @key_barcodes and (guide_target in @singles) (key = guide_target)

Computing normalized average expression matrices...


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))

groupby: index in @key_barcodes and (guide_target in @singles) (key = guide_target)

Computing clusters...


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))

groupby: index in @key_barcodes and (guide_target in @singles) (key = guide_target)

Generating summary statistics...
Done.


In [80]:
for col in np.setdiff1d(mean_pop.cells.columns, single_mean_pop.cells.columns):
    single_mean_pop.cells[col] = mean_pop.cells[col]

In [81]:
single_mean_pop.to_hdf('/home/tmn/sequencing/gi/gi/outs/single_mean_pop_with_normalized.hdf', store_normalized_matrix=True)

Writing matrix...
Writing normalized matrix...
Writing metadata...
Done in 0.107188940048s.
